scan for commands facebook?


In [5]:

function getPermissions() {
    return {
        'google contacts.ipynb[google contact settings]': 'Inner Circle'
    };
}

function scanCommandsFacebook() {
    return client
        .then(() => loginFacebook())
        .getUrl()
        .then(url => url.indexOf('https://www.facebook.com/messages/?filter=unread') === -1
             ? client.url('https://www.facebook.com/messages/?filter=unread')
             : [])
        .pause(500)
        .then(() => listFacebookThreads(false))
        .then(threads => {
            return importer.runAllPromises(threads.map(thread => resolve => {
                return readFacebookThread(thread)
                    .catch(e => console.log(e))
                    .then(r => resolve(r))
            }))
        })
        .then(friends => friends.filter(f => f.messages.filter(m => m.message.match(/^\s*megamind/ig)).length > 0))
        .catch(e => console.log(e))
}
module.exports = scanCommandsFacebook;


[Function: scanCommandsFacebook]

In [7]:
var importer = require('../Core');
var runSeleniumCell = importer.import('selenium cell');
var scanCommandsFacebook, getSettings;

$$.async();
runSeleniumCell([
    'log in facebook',
    'scrape facebook profile',
    'list facebook threads',
    'messages facebook',
    'scan commands facebook',
    'google contact settings',
])
    .then(r => {
        scanCommandsFacebook = r.scanCommandsFacebook;
        getSettings = r.getSettings;
        return scanCommandsFacebook()
    })
    .then(friends => {
        return importer.runAllPromises(friends.map(f => resolve => {
            getSettings(f.participants[0].name.split('\n')[0])
                .catch(console.log(e))
                .then(r => resolve(r))
        }))
    })
    .then(r => $$.sendResult(r))
    .catch(e => $$.sendError(e))


Initializing webdriver on localhost
[ 1511587716597, 'bd9165cacc918c80b58e1573bbac7a92' ]
[22:38:10]  COMMAND	GET 	 "/wd/hub/session/bd9165cacc918c80b58e1573bbac7a92/window_handle"
[22:38:10]  COMMAND	POST 	 "/wd/hub/session/bd9165cacc918c80b58e1573bbac7a92/window"
[22:38:10]  COMMAND	GET 	 "/wd/hub/session/bd9165cacc918c80b58e1573bbac7a92"
[ 1511587993349, '3133fe0feb95113004265446032ad1cb' ]
[22:38:10]  COMMAND	GET 	 "/wd/hub/session/3133fe0feb95113004265446032ad1cb/window_handle"
[22:38:10]  COMMAND	POST 	 "/wd/hub/session/3133fe0feb95113004265446032ad1cb/window"
[22:38:10]  COMMAND	GET 	 "/wd/hub/session/3133fe0feb95113004265446032ad1cb"
[ 'bd9165cacc918c80b58e1573bbac7a92',
  '3133fe0feb95113004265446032ad1cb' ]
[22:38:10]  COMMAND	GET 	 "/wd/hub/session/bd9165cacc918c80b58e1573bbac7a92"
bd9165cacc918c80b58e1573bbac7a92
[22:38:10]  COMMAND	GET 	 "/wd/hub/session/3133fe0feb95113004265446032ad1cb/window_handle"
[22:38:10]  COMMAND	POST 	 "/wd/hub/session/3133fe0feb95113004265446032a

[]

sync facebook threads?



In [ ]:
var importer = require('../Core');
var fs = require('fs');
var runSeleniumCell = importer.import('selenium cell');
var loginFacebook, listFacebookThreads, readFacebookThread;

var PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE;
var project = PROFILE_PATH + '/Conversations';

$$.async();
runSeleniumCell([
    'log in facebook',
    'list facebook threads',
    'messages from facebook',
])
    .then(r => ({
        loginFacebook, listFacebookThreads, readFacebookThread
    } = r).loginFacebook())
    .then(threads => {
        try { threads = JSON.parse(fs.readFileSync(project + '/facebook-threads.json')); }
        catch (e) { threads = []; }
    // TODO: compare glob/stat to see which threads need updating
        return importer.runAllPromises(threads.slice(450, 600)
            .map(t => ((resolve) => readFacebookThread(t).then((r) => resolve(r)))));
    })
    // TODO: when threads gets to 100% call listFacebookThreads()
    //.then(() => listFacebookThreads())


List Facebook threads?



In [ ]:
var PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE;
var project = PROFILE_PATH + '/Conversations';

function getFacebookThreads(threads) {
    return client
        .getAllUntil('//*[contains(@class, "scrollable")]',
                     '//*[contains(@data-href, "messages")]/@data-href',
                     threads,
                     (a, b) => a === b,
                     i => i < 20)
};

function listArchivedFacebook(threads) {
    return client
        .then(url => url.indexOf('/messages/archived') == -1
            ? client.url('https://www.facebook.com/messages/archived')
                .pause(1000)
                .click('.scrollable a[href="#"]')
            : Promise.resolve([]))
        .pause(2000)
        .then(() => getFacebookThreads(threads))
}

var threads, results;
function listFacebookThreads(threads, archived = false) {
    try {
        if(threads !== false) {
            threads = JSON.parse(fs.readFileSync(project + '/facebook-threads.json'));
        } else {
            threads = [];
        }
    }
    catch (e) {
        threads = [];
    }
    return client.getUrl()
        .then(url => url.indexOf('/messages/t') == -1
            ? client
                .click('[data-tooltip-content="Messages"]')
                .click('a[href*="/messages/t"]')
            : Promise.resolve([]))
        .pause(2000)
        .then(() => getFacebookThreads(threads))
        .then(r => threads = r)
        .then(() => archived ? listArchivedFacebook(threads) : threads)
        // list all message threads
        .then(r => {
            threads = r;
            fs.writeFileSync(
                project + '/facebook-threads.json',
                JSON.stringify(r, null, 4));
            return threads;
        })
        .catch(e => console.log(e))
};
module.exports = listFacebookThreads;
listFacebookThreads;



Get messages from facebook?



In [ ]:
var importer = require('../Core');
var chrono = require('chrono-node');
var glob = require('glob');
var fs = require('fs');

var PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE;
var project = PROFILE_PATH + '/Conversations';

function switchToParticipantThread(i) {
    return client
        .pause(1000)
        .click('//*[contains(@class, "scrollable")]//h4[contains(., "People")]/parent::*//li[' + i + ']')
        .pause(1000)
        .then(() => readFacebookProfileInfo())
        .catch(e => console.log(e))
};

function getThreadParticipants(thread) {
    return client
        .execute(() => {
            return document.evaluate(
                'count(//*[contains(@class, "scrollable")]//h4[contains(., "People")]/parent::*//li)',
                document, null,
                XPathResult.NUMBER_TYPE, null).numberValue;
        })
        .then(r => {
            var result = [];
            for (var i = 1; i < Math.min(3, r.value); i++) {
                result[result.length] = (i =>
                    resolve => {
                        var profile = {};
                        switchToParticipantThread(i)
                            .then(r => profile = r)
                            .url(thread)
                            .pause(3000)
                            .catch(e => console.log(e))
                            .then(() => resolve(profile))
                    })(i);
            }
            if (r.value === 0) {
                result[result.length] = (resolve => {
                    var profile = {};
                    readFacebookProfileInfo()
                        .then(r => profile = r)
                        .url(thread)
                        .pause(3000)
                        .then(() => resolve(profile))
                        .catch(e => console.log(e))
                })
            }
            return importer.runAllPromises(result).catch(e => console.log(e));
        })
        .catch(e => console.log(e))
};

function readFacebookMessages(messages, i = 0) {
    // TODO: add check for needing to go to LinkedIn
    // TODO: add check for needing to login
    return client
        .execute(() => {
            var result = [];
            var groups = document.getElementsByClassName('scrollable')[2];
            var times = groups.getElementsByTagName('h4');
            for (var t = 0; t < times.length; t++) {
                var bubble = times[t];
                while ((bubble = bubble.nextSibling) && bubble.nodeName === 'DIV') {
                    var from = bubble.getElementsByTagName('h5')[0];
                    result[result.length] = {
                        from: typeof from !== 'undefined' ? from.textContent : '',
                        time: times[t].innerText,
                        message: bubble.innerText
                    };
                }
            }
            return result;
        })
        .then(r => {
            var newMessages = r.value
                .filter(e => messages
                    .filter(m => m.message === e.message).length === 0);
            messages = newMessages.concat(messages);
            return newMessages.length > 0
                ? scrollClient('//*[contains(@role, "main")]//*[contains(@class, "scrollable")][1]', true, i)
                    .pause(2000)
                    .then(() => i > 3
                        ? messages
                        : readFacebookMessages(messages, i + 1))
                : Promise.resolve(messages);
        })
        .catch(e => console.log(e))
};

function readFacebookThread(thread, messages) {
    var threadId = thread.replace(/^\/|\/$/ig, '').split('/').pop()
        .replace(/[^a-z0-9]/ig, '_');
    var file = glob.sync('**/' + threadId + '-*.json', {cwd: project})[0];
    try {
        messages = JSON.parse(fs.readFileSync(file))
    }
    catch (e) {
        messages = []
    }
    var participants;
    return client.getUrl().then(url => url.indexOf(thread) === -1
        ? client.url(thread)
        : client)
    // get participants from topcard
        .pause(1000)
        .then(() => getThreadParticipants(thread))
        .then(p => (participants = p))

        // TODO: save to contacts
        .then(() => readFacebookMessages(messages || []))
        .then(messages => ({
            thread: thread,
            participants: participants || [],
            messages: messages.map(m => {
                var newTime = chrono.parseDate(m.time);
                newTime.setHours(newTime.getHours() - (new Date()).getTimezoneOffset() / 60)
                return Object.assign(m, {time: newTime})
            })
        }))
        .then(t => {
            var filename = project + '/'
                + threadId
                + '-' + t.participants
                    .map(p => p.name).join('')
                    .replace(/[^a-z0-9]/ig, '_') + '.json';
            fs.writeFileSync(filename, JSON.stringify(t, null, 4));
            return t;
        })
        .catch(e => console.log(e))
};
module.exports = readFacebookThread;


send facebook message?



In [ ]:
function sendFacebookMessage(message) {
    return client
        .click('//*[contains(@aria-label, "Type a message")]')
//        .keys(['Control', 'A', 'NULL'])
//        .keys('\uE127')
        .keys(message)
        .keys(' Mm')
        .keys('\uE007')
        .catch(e => console.log(e))
}
module.exports = sendFacebookMessage;
